In [1]:
import sys
sys.path.append("/Users/Mike/Desktop/UCSB/CS/cs111/CS111-2021-fall/Python") 

import os
import time
import math
import autograd.numpy as np
import autograd.numpy.linalg as npla
from autograd import elementwise_grad, value_and_grad, grad
from scipy.optimize import minimize
from collections import defaultdict
from itertools import zip_longest
from functools import partial
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import numpy as np
import networkx as nx
import json
from tabulate import tabulate

import cs111

import matplotlib
%matplotlib tk
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
from matplotlib import animation

np.set_printoptions(precision = 4)

## Problem 1

In [3]:
tolerances = [1e-3, 1e-7, 1e-15]    # precision to 4, 8, and 16 digits
num_of_decimals = [3, 7, 15]

def newtons_method_sqrt(value, x0):
    # print the value we are estimating, the correct result,
    print("\n--------------------------------------------------------\n",\
          "Estimating the square root of",\
          value, " starting at ", x0,\
          ":\n--------------------------------------------------------\n")
    # we do this for every tolerance
    for i in range(len(tolerances)):
        correct_xk = np.sqrt(value)
        precision = num_of_decimals[i] # only show digits that matter given the tolerance
        # precision = 15 # show 15 decimal places
        data = []       # a data list that'll hold our findings
        xk, k = x0, 0   # set the initial xk and counter variables
        # every iteration, we will add a row of 
        # information to the data array
        while(tolerances[i] < abs(round(xk, num_of_decimals[i]) - correct_xk)):
            # record the [k, kx, and kx**2] of this step
            data.append([k, xk, xk**2.0])
            # take the iterative step
            k += 1
            xk = (0.5)*(xk + value/xk)
        # record the last iteration's information
        data.append([k, xk, xk**2])
        # set the floatfmt string using tolerance and
        # print the table for every tolerance for the given value
        my_floatfmt = "." + str(precision) + "f"
        print("tolerance =", tolerances[i], "\n",\
                tabulate(data, headers=["k", "xk", "xk**2", "correct xk"],\
                tablefmt="grid", floatfmt=my_floatfmt),\
                "\nsteps taken:", k, "\n")
    # to let us know we are done estimating
    # the value for each necessary tolerance
    print("Done estimating the value", value)

In [4]:
newtons_method_sqrt(4.0, 1.0)
newtons_method_sqrt(2.0, 1.0)


--------------------------------------------------------
 Estimating the square root of 4.0  starting at  1.0 :
--------------------------------------------------------

tolerance = 0.001 
 +-----+-------+---------+
|   k |    xk |   xk**2 |
+=====+=======+=========+
|   0 | 1.000 |   1.000 |
+-----+-------+---------+
|   1 | 2.500 |   6.250 |
+-----+-------+---------+
|   2 | 2.050 |   4.202 |
+-----+-------+---------+
|   3 | 2.001 |   4.002 |
+-----+-------+---------+ 
steps taken: 3 

tolerance = 1e-07 
 +-----+-----------+-----------+
|   k |        xk |     xk**2 |
+=====+===========+===========+
|   0 | 1.0000000 | 1.0000000 |
+-----+-----------+-----------+
|   1 | 2.5000000 | 6.2500000 |
+-----+-----------+-----------+
|   2 | 2.0500000 | 4.2025000 |
+-----+-----------+-----------+
|   3 | 2.0006098 | 4.0024394 |
+-----+-----------+-----------+
|   4 | 2.0000001 | 4.0000004 |
+-----+-----------+-----------+ 
steps taken: 4 

tolerance = 1e-15 
 +-----+-------------------+----

## Problem 2

#### 2.1
Here's the function $f(x)$ that we have to find the argmin{} of:
$$f(x) = \frac{\sqrt{x^2 + 20^2}}{5} + \sqrt{(80-x)^2 + 40^2}$$

In [122]:
def f(x):
    first = np.sqrt(x**2.0 + 20.0**2.0) / 5.0
    second = np.sqrt((80.0 - x)**2.0 + 40.0**2.0)
    return first + second

In [123]:
result = scipy.optimize.minimize(f, 80.0)

The above tells us that the lifeguard should run to $$(72.1435, 0),$$ and then swim to the swimmer. The result above also tells us that $$f(72.1435) = 55.73714459499687 \text{ seconds},$$ which is the number of seconds that it'll take the lifeguard to get the swimmer along the straight shoreline.

In [127]:
# modify our f to give the distances so we can plot them on the chart
def modified_f(x):
    first = np.sqrt(x**2.0 + 20.0**2.0) / 5.0
    second = np.sqrt((80.0 - x)**2.0 + 40.0**2.0)
    return (first * 5.0, second)
modified_f(72.1435) # first term is the running distance,
                    # second term is the swimming distance
plt.xlim([0, 100]) # x limits
plt.ylim([-60, 40]) # y limits
plt.xlabel( "Distance Along the Shoreline" )
plt.ylabel( "Distance from the Oceanline" )
plt.title( "Optimal Path for a Lifeguard Along a Straight Shoreline" )

# this is the lifeguard
plt.plot([0], [20], marker="o", markersize=10,\
         markeredgecolor="red", markerfacecolor="green")
plt.text(0 + 3, 20 + 3, 'Lifeguard') # label
# this is the swimmer
plt.plot([80], [-40], marker="o", markersize=10,\
         markeredgecolor="blue", markerfacecolor="green")
plt.text(80 + 3, -40 + 3, 'Swimmer') # label
# the spot the lifeguard should run to
plt.plot([result['x'][0]], [0], marker="x", markersize=10,\
         markeredgecolor="black", markerfacecolor="black")
plt.text(result['x'][0] - 17, 0 + 8, str('the point x at ('\
                                         + str(round(result['x'][0], 4))\
                                         + ',0)'))# label
# now we need a line from the lifeguard to the x
lifeguard, x = [0, result['x'][0]], [20, 0]
plt.plot(lifeguard, x, label = str('running for '\
                                   + str(modified_f(result['x'][0])[0])\
                                   + ' meters')) # plot
# and we need a line from the x to the swimmer
x, swimmer = [result['x'][0], 80], [0, -40]
plt.plot(x, swimmer, label = str('swimming for '\
                                 + str(modified_f(result['x'][0])[1])\
                                 + ' meters')) # plot
plt.axhline(0, color='black', label = 'straight shoreline') # draw the shoreline
plt.legend()
plt.show() # show the plot

In [128]:
plt.close('all')

#### 2.2
Here's the function $f(x)$ that we have to find the argmin{} of:
$$f(x) = \frac{\sqrt{x^2 + (y+20)^2}}{5} + \sqrt{(80-x)^2 + (-40-y)^2},$$
where
$$y = - \sqrt{1000 - \frac{x^2}{10}}$$

In [142]:
def y(x):
    return -np.sqrt(1000.0 - (x**2.0) / 10.0)
def f(x):
    first = np.sqrt(x**2.0 + (y(x) + 20.0)**2.0) / 5.0
    second = np.sqrt((80.0 - x)**2.0 + (-40.0 - y(x))**2.0)
    return first + second

In [143]:
result = scipy.optimize.minimize(f, 80.0)
result

      fun: 34.08474202400345
 hess_inv: array([[16.3784]])
      jac: array([4.7684e-07])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 4
     njev: 6
   status: 0
  success: True
        x: array([70.5215])

In [144]:
y(result['x'][0])

-22.420351359105403

The above tells us that the lifeguard should run to $$(70.5215, -22.420343524910585),$$ and then swim to the swimmer. The result above also tells us that $$f(70.5215) = \text{34.08474202400345} \text{ seconds},$$ which is the number of seconds that it'll take the lifeguard to get the swimmer along the curved shoreline.

In [147]:
# modify our f to give the distances so we can plot them on the chart
def modified_f(x):
    first = np.sqrt(x**2.0 + (y(x) + 20.0)**2.0) / 5.0
    second = np.sqrt((80.0 - x)**2.0 + (-40.0 - y(x))**2.0)
    return (first * 5.0, second)

modified_f(70.5215) # first term is the running distance,
                    # second term is the swimming distance
plt.xlim([0, 100]) # x limits
plt.ylim([-60, 40]) # y limits
plt.xlabel( "Distance Along the Shoreline" )
plt.ylabel( "Distance from the Oceanline" )
plt.title( "Optimal Path for a Lifeguard Along a Curved Shoreline" )
# this is the lifeguard
plt.plot([0], [20], marker="o", markersize=10,\
         markeredgecolor="red", markerfacecolor="green")
plt.text(0 + 3, 20 + 3, 'Lifeguard') # label
# this is the swimmer
plt.plot([80], [-40], marker="o", markersize=10,\
         markeredgecolor="blue", markerfacecolor="green")
plt.text(80 + 3, -40 + 3, 'Swimmer') # label
# the spot the lifeguard should run to
plt.plot([result['x'][0]], [y(result['x'][0])], marker="x", markersize=10,\
         markeredgecolor="black", markerfacecolor="black")
plt.text(result['x'][0] - 23, y(result['x'][0]) + 12, str('the point x at ('\
                                         + str(round(result['x'][0], 4))\
                                         + ',' + str(round(y(result['x'][0]), 4)) + ')')) # label
# now we need a line from the lifeguard to the x
lifeguard, x = [0, result['x'][0]], [20, y(result['x'][0])]
plt.plot(lifeguard, x, label = str('running for '\
                                   + str(modified_f(result['x'][0])[0])\
                                   + ' meters')) # plot
# and we need a line from the x to the swimmer
x, swimmer = [result['x'][0], 80], [y(result['x'][0]), -40]
plt.plot(x, swimmer, label = str('swimming for '\
                                 + str(modified_f(result['x'][0])[1])\
                                 + ' meters')) # plot
# display the x-axis
plt.axhline(0, color= 'black')
# draw the curved shoreline
x_vals = np.linspace(0,100,100)
y_vals = np.array([y(x_vals[i]) for i in range(len(x_vals))])
plt.plot(x_vals, y_vals, color = 'black', label = 'curved shoreline')

plt.legend()
plt.show() # show the plot

In [2]:
plt.close("all")

In [ ]:
## quiz 9
tolerances = [1e-3, 1e-7, 1e-15]    # precision to 4, 8, and 16 digits
num_of_decimals = [3, 7, 15]

def newtons_method_sqrt(value, x0):
    # print the value we are estimating, the correct result,
    print("\n--------------------------------------------------------\n",\
          "Estimating the power of 4 of",\
          value, " starting at", x0,\
          ":\n--------------------------------------------------------\n")
    # we do this for every tolerance
    for i in range(len(tolerances)):
        correct_xk = value**4.0
        precision = num_of_decimals[i] # only show digits that matter given the tolerance
        # precision = 15 # show 15 decimal places
        data = []       # a data list that'll hold our findings
        xk, k = x0, 0   # set the initial xk and counter variables
        # every iteration, we will add a row of 
        # information to the data array
        while(tolerances[i] < abs(round(xk, num_of_decimals[i]) - correct_xk)):
            # record the [k, kx, and kx**2] of this step
            data.append([k, xk, np.sqrt(np.sqrt(xk))])
            # take the iterative step
            k += 1
            xk = (0.5)*(xk + value/xk)
        # record the last iteration's information
        data.append([k, xk, xk**2])
        # set the floatfmt string using tolerance and
        # print the table for every tolerance for the given value
        my_floatfmt = "." + str(precision) + "f"
        print("tolerance =", tolerances[i], "\n",\
                tabulate(data, headers=["k", "xk", "xk**2", "correct xk"],\
                tablefmt="grid", floatfmt=my_floatfmt),\
                "\nsteps taken:", k, "\n")
    # to let us know we are done estimating
    # the value for each necessary tolerance
    print("Done estimating the value", value)

In [4]:
def f(x):
    return x**4.0

scipy.optimize.newton(f, 2.0)

RuntimeError: Failed to converge after 50 iterations, value is 8.738052344719754e-05.